In [50]:
import requests
import bs4 as soup

In [51]:
request_headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Cookie": "JSESSIONID=2F6E7BF66D9BAE0846E5313DBFDEA42B.tomcat-13409",
    "Host": "lsu.ent.sirsi.net",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    }

In [52]:
url = 'https://lsu.ent.sirsi.net/client/en_US/login.loginform'
data = {'j_username': '',
        'j_password': '',
        't:formdata': ''}

In [53]:
s = requests.Session()
r = s.post(url, data=data)

In [ ]:
def get_max_search_fields_page():
    search_fields_page = s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managefields')
    search_soup = soup.BeautifulSoup(search_fields_page.content, 'lxml')
    pages = search_soup.select('.t-data-grid-pager a')
    max_page = max(int(i.text) for i in pages)
    return max_page

def get_search_fields_items(page, search_fields_dict=None):
    search_fields_page = s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managefields.grid.pager/{}'.format(page))
    search_soup = soup.BeautifulSoup(search_fields_page.content, 'lxml')
    search_field_rows = search_soup.tbody
    if not search_fields_dict:
        search_fields_dict = dict()
    for row in search_soup.tbody:
        search_field_code, search_field_name, search_field_hierarchy, links = row.contents
        if search_field_code.string in search_fields_dict:
            print('duplicate search field code found {}'.format(search_field_code))
            break
        search_fields_dict[search_field_code.string] = {'code': search_field_code.string, 
                                           'name': search_field_name.string,
                                           'hierarchy': search_field_hierarchy.string.strip(),
                                           }
    return search_fields_dict
        
def get_search_fields_editfield_page(search_field_code):
    search_field_code = search_field_code.replace('/', '$002f').replace(' ', '$0020')
    return s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managefields.edit/{}'.format(search_field_code))

def get_search_fields_marc_page(search_field_code):
    search_field_code = search_field_code.replace('/', '$002f').replace(' ', '$0020')
    return s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managefields.confmarcmap/{}'.format(search_field_code))

In [ ]:
def build_basic_search_dict():
    search_fields_dict = None
    max_page = get_max_search_fields_page()
    for page_num in range(1, max_page+1):
        search_fields_dict = get_search_fields_items(page_num, search_fields_dict)
    return search_fields_dict

In [ ]:
s.cookies

In [ ]:
search_fields_dict = build_basic_search_dict()

In [ ]:
search_field_edit_example = get_edit_search_fields_page('BIBLIOGRAPHY504')

In [ ]:
edit_soup = soup.BeautifulSoup(search_field_edit_example.content, 'lxml')

In [54]:
def parse_checkbox(css_id, my_soup):
    try:
        css_code = '#{}'.format(css_id)
        my_soup.select(css_code)[0]['checked']
        return True
    except KeyError:
        return False
    except IndexError:
        return "Unknown"

def parse_dropdown(css_id, my_soup):
    try:
        css_code = '#{}'.format(css_id)
        return my_soup.select(css_code)[0].select('[selected]')[0].string
    except KeyError:
        return "None"
    except IndexError:
        return "None"

def parse_textbox(css_id, my_soup):
    try:
        css_code = '#{}'.format(css_id)
        return my_soup.select(css_code)[0]['value']
    except KeyError:
        return "None"
    except IndexError:
        return "None"

def add_search_field_editfield_dict(search_fields_dict):
    for search_field in search_fields_dict:
        editfield_page = get_search_fields_editfield_page(search_field)
        editfield_soup = soup.BeautifulSoup(editfield_page.content, 'lxml')
        if not search_fields_dict[search_field].get('EditField'):
            search_fields_dict[search_field]['EditField'] = dict()
        search_fields_dict[search_field]['EditField']['descripton'] = editfield_soup.select('#description')[0].string
        search_fields_dict[search_field]['EditField']['hierarchy'] = parse_dropdown('hierarchy', editfield_soup)
        search_fields_dict[search_field]['EditField']['fieldCodeType'] = parse_dropdown('fieldCodeType', editfield_soup)
        search_fields_dict[search_field]['EditField']['indexed'] = parse_checkbox('indexed', editfield_soup)
        search_fields_dict[search_field]['EditField']['normalizer'] = parse_dropdown('normalizer', editfield_soup)
        search_fields_dict[search_field]['EditField']['dataType'] = parse_dropdown('dataType', editfield_soup)
        search_fields_dict[search_field]['EditField']['sort'] = parse_dropdown('sort', editfield_soup)
        search_fields_dict[search_field]['EditField']['repeatable'] = parse_checkbox('repeatable', editfield_soup)
        search_fields_dict[search_field]['EditField']['splitonspace'] = parse_checkbox('splitonspace', editfield_soup)
        search_fields_dict[search_field]['EditField']['facet'] = parse_checkbox('facet', editfield_soup)
        search_fields_dict[search_field]['EditField']['exclusiveFacet'] = parse_checkbox('exclusiveFacet', editfield_soup)
        search_fields_dict[search_field]['EditField']['displayFieldType'] = parse_dropdown('displayFieldType', editfield_soup)
        search_fields_dict[search_field]['EditField']['hitlistAvail'] = parse_checkbox('hitlistAvail', editfield_soup)
        search_fields_dict[search_field]['EditField']['detailAvail'] = parse_checkbox('detailAvail', editfield_soup)
        search_fields_dict[search_field]['EditField']['detailTab'] = parse_dropdown('detailTab', editfield_soup)
        search_fields_dict[search_field]['EditField']['specialProcessing'] = parse_dropdown('specialProcessing', editfield_soup)

def add_search_field_configuremarcmap_dict(search_fields_dict):
    for search_field in search_fields_dict:
        configuremarcmap_page = get_search_fields_marc_page(search_field)
        configuremarcmap_soup = soup.BeautifulSoup(configuremarcmap_page.content, 'lxml')
        search_fields_dict[search_field]['ConfigureMarcMap'] = parse_dropdown('marcMap', configuremarcmap_soup)
        

In [ ]:
add_search_field_configuremarcmap_dict(search_fields_dict)

In [ ]:
add_search_field_editfield_dict(search_fields_dict)

In [ ]:
search_fields_dict

In [61]:
def get_max_marc_maps_page():
    search_fields_page = s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managemarcmaps')
    search_soup = soup.BeautifulSoup(search_fields_page.content, 'lxml')
    pages = search_soup.select('.t-data-grid-pager a')
    max_page = max(int(i.text) for i in pages)
    return max_page

def get_marc_maps_editfield_page(search_field_code):
    search_field_code = str(search_field_code).replace('/', '$002f').replace(' ', '$0020')
    return s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managemarcmaps.edit/{}'.format(search_field_code))

def get_marc_maps_marc_page(search_field_code):
    search_field_code = str(search_field_code).replace('/', '$002f').replace(' ', '$0020')
    return s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managemarcmaps.confmarc21tags/{}/false'.format(search_field_code))

def get_marc_maps_subfields_page(search_field_code):
    search_field_code = str(search_field_code).replace('/', '$002f').replace(' ', '$0020')
    return s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managemarctags.configuresubfields/{}'.format(search_field_code))

def get_max_marc_maps_subfields(subfields_soup):
    return len(subfields_soup.select('.t-data-grid tbody tr'))

In [56]:
get_max_marc_maps_page()

10

In [65]:
response = get_marc_maps_editfield_page('TITLE ALL')
response = get_marc_maps_marc_page('TITLE ALL')
fields = soup.BeautifulSoup(response.content, 'lxml').select('#tagsTable tbody tr')
item_marc_dict = dict()
for num, i in enumerate(fields):
    field_num = i.td.string.strip()
    response = s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managemarctags.edit/{}'.format(num))
    field_soup = soup.BeautifulSoup(response.content, 'lxml')
    tag = parse_textbox('tag', field_soup)
    field_marc_dict = {
        'tag': tag,
        'marcCond': parse_textbox('marcCond', field_soup),
        'firstPrefix': parse_textbox('firstPrefix', field_soup),
        'firstSuffix': parse_textbox('firstSuffix', field_soup),
        'otherPrefix': parse_textbox('otherPrefix', field_soup),
        'otherSuffix': parse_textbox('otherSuffix', field_soup),
        'concatSubfields': parse_checkbox('concatSubfields', field_soup),
        'concatTags': parse_checkbox('concatTags', field_soup),
        'fixedField': parse_checkbox('fixedField', field_soup),
        'offset': parse_textbox('offset', field_soup),
        'length': parse_textbox('length', field_soup),
        }
    item_marc_dict[tag] = field_marc_dict
    item_marc_dict['subfields'] = dict()
    response = get_marc_maps_subfields_page(num)
    subfields_soup = soup.BeautifulSoup(response.content)
    max_subfields = get_max_marc_maps_subfields(subfields_soup)
    for count in range(0, max_subfields):
        subfield_page = s.get('https://lsu.ent.sirsi.net/client/en_US/admin/search/managemarcsubfields.edit/{}'.format(count))
        subfield_page_soup = soup.BeautifulSoup(subfield_page.content)
        subfield = parse_textbox('subfield', subfield_page_soup)
        subfield_marc_dict = {
            'subfield': subfield,
            'beginningPunct': parse_textbox('beginningPunct', subfield_page_soup),
            'endingPunct': parse_textbox('endingPunct', subfield_page_soup),
            'marcCond': parse_textbox('marcCond', subfield_page_soup),
        }
        item_marc_dict['subfields'][subfield] = subfield_marc_dict
print(item_marc_dict)

{'245': {'tag': '245', 'marcCond': 'None', 'firstPrefix': 'None', 'firstSuffix': 'None', 'otherPrefix': 'None', 'otherSuffix': 'None', 'concatSubfields': True, 'concatTags': False, 'fixedField': False, 'offset': '0', 'length': '0'}, 'subfields': {'a': {'subfield': 'a', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}, 'b': {'subfield': 'b', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}, 'f': {'subfield': 'f', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}, 'k': {'subfield': 'k', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}, 'n': {'subfield': 'n', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}, 'p': {'subfield': 'p', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}, 's': {'subfield': 's', 'beginningPunct': 'None', 'endingPunct': 'None', 'marcCond': 'None'}}, '100': {'tag': '100', 'marcCond': 'None', 'firstPrefix': 'None', 'firstSuffix': 'None', 'otherPrefix'

In [31]:
subfield_soup = soup.BeautifulSoup(response.content, 'lxml')

In [42]:
subfield_soup.select('.t-data-grid tbody tr')[0].next_element.string


'a'

In [60]:
len(max_subfields.select('.t-data-grid tbody tr'))


7

In [ ]:

#     search_soup.tbody.findall('td')